# Prerequisites

1) Using python 3.8 install requirements.txt in a virtual environment (run [installation.sh](installation.sh) on unix or [installation.bat](installation.bat) on windows)
2) Have a Neo4j database with APOC and n10s libraries available. See [here](https://github.com/GSK-Biostatistics/neointerface) for introduction to Neo4j and [here](https://github.com/GSK-Biostatistics/neointerface/blob/main/Appendixes.md) for best ways to deploy it
3) Set environment variables to enable connection to your neo4j database. The following are required:
NEO4J_HOST, NEO4J_USER, NEO4J_PASSWORD, NEO4J_RDF_HOST  
You may opt to populate the template in setenv.sh (on unix) or setenv.bat(on windwows) and run it. Remember not to push any of your credentials to GitHub.  
4) Set environment variable to enable connection to CDISC API: CDISC_LIB_KEY (you can obtain the key from your profile after setting up and account on CDISC Library: https://api.developer.library.cdisc.org/profile)  

When the above is done you should be ready to run this notebook with an example.

In [1]:
import os
import time

# Downloading CDISC pilot study data
In order to demonstrate clinical data ingestion into a graph we will download some data from [phuse-org repository](https://github.com/phuse-org/phuse-scripts/tree/master/data/sdtm/cdiscpilot01)  
For demonstration purposes we will only load 2 domains, feel free to experiment loading other domains as well.

In [2]:
from utils.utils import get_cdisc_pilot_data
domains = ['DM', 'DS']
#domains = ['DM', 'EX', 'AE', 'LB', 'VS', 'DS']
downloads = get_cdisc_pilot_data(domains)
print("Done: ", downloads['ok'])
if (downloads['error']):
    print("Failed to download some files:")
    print(downloads['error'])


--------
Getting domain: DM
Download from url: https://github.com/phuse-org/phuse-scripts/raw/master/data/sdtm/cdiscpilot01/dm.xpt

--------
Getting domain: DS
Download from url: https://github.com/phuse-org/phuse-scripts/raw/master/data/sdtm/cdiscpilot01/ds.xpt
Done:  [{'folder': 'temp/data/sdtm/cdiscpilot01', 'file': 'dm.xpt'}, {'folder': 'temp/data/sdtm/cdiscpilot01', 'file': 'ds.xpt'}]


# The next step is to load tabular data "as is" into the graph database
For details on FileDataLoader [see](https://github.com/GSK-Biostatistics/tab2neo/tree/main/data_loaders)

In [3]:
from data_loaders.file_data_loader import FileDataLoader
fdl = FileDataLoader()
fdl.clean_slate()
for download in downloads['ok']:
    df = fdl.load_file(download['folder'], download['file'])
    print(f"{download['file']} loaded")

---------------- Initializing NeoInterface -------------------
Connection to bolt://localhost:27687 established
 --- Deleting all nodes in the database ---
dm.xpt loaded
ds.xpt loaded


# Creating Class-Relationship model from data

In [4]:
from model_managers.model_manager import ModelManager
mm = ModelManager()
mm.create_model_from_data()

---------------- Initializing NeoInterface -------------------
Connection to bolt://localhost:27687 established
---------------- <class 'model_managers.model_manager.ModelManager'> initialized -------------------


# Enriching model from data with CDISC labels and higher-level classes
## Using CDISC API to load metadata into neo4j 

In [5]:
from cdisc_model_managers.cdisc_api import CDISCAPI

cdiscapi = CDISCAPI()
#res = cdiscapi.query("/mdr/products")

#loading sdtm model
model_href = "/mdr/sdtm/2-0"
model_label = "SDTM Model"
sdtm_model = cdiscapi.query(model_href)
sdtm_model['href'] = model_href
mm.query(f"MATCH (root:`{model_label}`)-[*1..3]->(x) DETACH DELETE x, root") #cleanup
print(f"Loading and unpacking {model_href} into Neo4j with root node {model_label}")
mm.load_dict(sdtm_model, label=model_label,  maxdepth=3)

#loading sdtm ig
ig_href = "/mdr/sdtmig/3-4"
ig_label = "SDTM IG"
sdtmig = cdiscapi.query(ig_href)
sdtmig['href'] = ig_href
mm.query(f"MATCH (root:`{ig_label}`)-[*1..4]->(x) DETACH DELETE x, root") #cleanup
print(f"Loading and unpacking {ig_href} into Neo4j with root node {ig_label}")
mm.load_dict(sdtmig, label=ig_label,  maxdepth=4)

# #merging same things;
# mm.query("MATCH (x:`SDTM Model`), (y) WHERE y:self or y:parentProduct call apoc.refactor.mergeNodes([x,y], {}) yield node RETURN node")

2022-12-18 13:01:07,959    cld    INFO    -------------------------------   Loaded CLD Logger    -------------------------------


Loading and unpacking /mdr/sdtm/2-0 into Neo4j with root node SDTM Model
Loading and unpacking /mdr/sdtmig/3-4 into Neo4j with root node SDTM IG


## Enriching model with parent classses (variables):

In [6]:
mm.query("""
MATCH (c:Class), (cv:classVariables)
WHERE c.short_label<>cv.name and cv.name contains ("--")
and size(apoc.text.regexGroups(c.short_label, "^" + apoc.text.replace(cv.name,"-",".") + "$")) > 0
MERGE (pc:Class{label:cv.label})
SET pc.short_label = cv.name
MERGE (c)-[:SUBCLASS_OF]->(pc)
MERGE (pc)-[:PROVENANCE]->(cv)
""")

[]

## Enriching model with parent classses (datasets):

In [7]:
mm.query("""
MATCH (c:Class), (ds:datasets)<-[:datasets]-(oc:classes) 
WHERE c.short_label = ds.name
SET c.label = ds.label
MERGE (pc:Class{label: oc.label, short_label:toUpper(oc.name)})
MERGE (c)-[:SUBCLASS_OF]->(pc)
MERGE (pc)-[:PROVENANCE]->(oc)
""")

[]

## Enriching model with long class labels:

In [8]:

res = mm.query("""
MATCH (c:Class), (dv:datasetVariables)
WHERE c.short_label = dv.name
WITH c, collect(distinct dv.label) as coll
SET c.label = 
    CASE c.short_label
        WHEN 'USUBJID' THEN 'Subject'
        WHEN 'STUDYID' THEN 'Study'
    ELSE
        CASE WHEN size(coll) = 1 THEN 
            coll[0]
        ELSE
            'ERROR: >1 label exists: ' + apoc.text.join(coll, ', ')
        END
    END
WITH *
WHERE c.label starts with "ERROR:"
RETURN *
""")
res

[]

# Applying graph model on the data(reshaping)


In [9]:
from model_appliers.model_applier import ModelApplier
ma = ModelApplier(mode="schema_CLASS")
ma.delete_reshaped(batch_size=10000) #deleting reshaped data in case it was reshaped previously

start_time = time.time()
ma.reshape_all()
print(
    f"Reshaping done in: {(time.time() - start_time):.2f} seconds\n"
    f"You may browse clinical data in a form of a graph using Neo4j Browser"
    f": {os.getenv('NEO4J_HOST').replace('neo4j', 'http').replace('7687','7474')}"
)

---------------- Initializing NeoInterface -------------------
Connection to bolt://localhost:27687 established
 ------------------ Deleting reshaped instances ------------------
[[{'total': 0, 'batches': 1, 'failedBatches': 0, 'failedOperations': 0}], [{'total': 0, 'batches': 1, 'failedBatches': 0, 'failedOperations': 0}]]
 ------ Refactoring loaded data per graph class_ definition.  EXECUTING PART 1 --------- 
 ------ Refactoring loaded data per graph class_ definition.  EXECUTING PART 2 --------- 
    LOOPING OVER  41  entries in helper list:
--------------- Creating IS_A relationship ----------------------
[[{'total': 7099, 'batches': 8, 'failedBatches': 0, 'failedOperations': 0}]]
--------------- Linking classes ----------------------
[[{'total': 15890, 'batches': 16, 'failedBatches': 12, 'failedOperations': 11890}], [{'total': 11890, 'batches': 12, 'failedBatches': 0, 'failedOperations': 0}]]
[{'total': 0, 'batches': 1, 'failedBatches': 0, 'failedOperations': 0}]
Reshaping done i

# Clean-up

In [10]:
# #In order to enable Methods to work with already existing Relationships (btw parent and a child's neighbour)
# #created during refactoring, we explicitly create Relationship nodes
mm.propagate_rels_to_parent_class()
# labels from the domains that were not loaded may be confusing
print("Removing auxilary term labels")
mm.remove_auxilary_term_labels()
print("Done")

Copying Relationships to 'parent' Classes where (child)-[:SUBCLASS_OF]->(parent)
Removing auxilary term labels
Done


# Checking reshaping worked as expected
Not that the data is in the form of a graph we can further use CLD modules to wrangle and query it.  
DataProvider enables a python interface to query the data from the model without a need to write Cypher queries:

In [11]:
from data_providers.data_provider import DataProvider
dp = DataProvider()
df = dp.get_data_generic(
    labels=['Subject', 'Disposition', 'Dictionary-Derived Term', 'Start Date/Time of Observation'], #entities
    where_map={'Dictionary-Derived Term': {'rdfs:label':'COMPLETED'}}, #where condition
    infer_rels=True,
    return_nodeid=False,
    use_shortlabel=True,
    use_rel_labels=True,
    return_propname=False
)
assert not(df.empty)
df

---------------- Initializing NeoInterface -------------------
Connection to bolt://localhost:27687 established
---------------- <class 'model_managers.model_manager.ModelManager'> initialized -------------------
---------------- Initializing NeoInterface -------------------
Connection to bolt://localhost:27687 established
---------------- <class 'data_providers.data_provider.DataProvider'> initialized -------------------


2022-12-18 13:03:23,594    cld    INFO    QUERY: MATCH (`DS`:`Disposition`),
(`USUBJID`:`Subject`),
(`--DECOD`:`Dictionary-Derived Term`),
(`--STDTC`:`Start Date/Time of Observation`),
(`DS`)-[`DS_DSDECOD_--DECOD`:`DSDECOD`]->(`--DECOD`),
(`DS`)-[`DS_DSSTDTC_--STDTC`:`DSSTDTC`]->(`--STDTC`),
(`DS`)-[`DS_USUBJID_USUBJID`:`USUBJID`]->(`USUBJID`)
WHERE `--DECOD`.`rdfs:label` = $par_1


RETURN apoc.map.mergeList([apoc.map.fromPairs([key in keys(CASE WHEN `USUBJID`{.*} IS NULL THEN {} ELSE `USUBJID`{.*} END) | ["USUBJID", CASE WHEN `USUBJID`{.*} IS NULL THEN {} ELSE `USUBJID`{.*} END[key]]])
, apoc.map.fromPairs([key in keys(CASE WHEN `DS`{.*} IS NULL THEN {} ELSE `DS`{.*} END) | ["DS", CASE WHEN `DS`{.*} IS NULL THEN {} ELSE `DS`{.*} END[key]]])
, apoc.map.fromPairs([key in keys(CASE WHEN `--DECOD`{.*} IS NULL THEN {} ELSE `--DECOD`{.*} END) | ["--DECOD", CASE WHEN `--DECOD`{.*} IS NULL THEN {} ELSE `--DECOD`{.*} END[key]]])
, apoc.map.fromPairs([key in keys(CASE WHEN `--STDTC`{.*} IS NULL

,--DECOD,--STDTC,USUBJID
0,COMPLETED,2014-07-02,01-701-1015
1,COMPLETED,2014-01-14,01-701-1028
2,COMPLETED,2014-12-30,01-701-1034
3,COMPLETED,2014-07-09,01-701-1097
4,COMPLETED,2014-09-09,01-701-1118
...,...,...,...
105,COMPLETED,2014-03-04,01-717-1446
106,COMPLETED,2013-11-17,01-718-1139
107,COMPLETED,2013-07-31,01-718-1150
108,COMPLETED,2014-01-09,01-718-1254
